## Geocoding
### Overview of Geocoders
Geocoding is the process of transforming place names or addresses into coordinates. In this lesson we will learn how to geocode addresses using Geopandas and geopy.

Geopy and other geocoding libaries (such as geocoder) make it easy to locate the coordinates of addresses, cities, countries, and landmarks across the globe using web services (“geocoders”). In practice, geocoders are often Application Programming Interfaces (APIs) where you can send requests, and receive responses in the form of place names, addresses and coordinates.

Geopy offers access to several geocoding services, including:
- ESRI ArcGIS

- Baidu Maps

- Bing

- GeocodeFarm

- GeoNames

- Google Geocoding API (V3)

- HERE

- IGN France

- Mapquest

- OpenCage

- OpenMapQuest

- Open Street Map Nominatim

- What3words

- Yandex

Check the geopy documentation for more details about how to use each service via Python.

As you see, there are plenty of geocoders where to choose from! The quality of the geocoders might differ depending on the underlying data. For example, some addresses might exists on OpenStreetMap, but not on Google Maps and thus they can be geocoded using the Nominatim geocoder, but not using the Google Geocoding API - and vice versa.

Geocoding services might require an API key in order to use them. (i.e. you need to register for the service before you can access results from their API). Furthermore, rate limiting also restrict the use of these services. The geocoding process might end up in an error if you are making too many requests in a short time period (eg. trying to geocode large number of addresses). See hints for dealing with rate limiting when geocoding pandas DataFrames from the geopy documentation. If you pay for the geocoding service, you can naturally make more requests to the API.

In this lesson we will use the Nominatim geocoder for locating a relatively small number of addresses. Usage of the Nominatim geocoding service is rate-limited to 1 request per second (3600 / hour). You can read more about Nominatim usage policy in here

Luckily, Nominatim, which is a geocoder based on OpenStreetMap data does not require a API key to use their service if it is used to geocode only a small number of addresses.

### Geocoding in Geopandas
It is possible to do geocoding in Geopandas through geopy. Geopandas has a function called geocode() that can geocode a list of addresses (strings) and return a GeoDataFrame containing the resulting point objects in geometry column.

Let’s try this out.

We will geocode addresses stored in a text file called addresses.txt. These addresses are located in the Helsinki Region in Southern Finland.

The first rows of the data look like this:

In [1]:
# Import necessary modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Filepath
fp = r"data/addresses.txt"

# Read the data
data = pd.read_csv(fp, sep=';')

In [2]:
data.head()

,id,addr
0,1000,"Itämerenkatu 14, 00101 Helsinki, Finland"
1,1001,"Kampinkuja 1, 00100 Helsinki, Finland"
2,1002,"Kaivokatu 8, 00101 Helsinki, Finland"
3,1003,"Hermannin rantatie 1, 00580 Helsinki, Finland"
4,1005,"Tyynenmerenkatu 9, 00220 Helsinki, Finland"


### Geocode addresses using Nominatim
Now we have our data in a pandas DataFrame and we can geocode our addresses using the geopandas geocoding function that uses geopy package in the background.

- Let’s import the geocoding function and geocode the addresses (column addr) using Nominatim.

- Remember to provide a custom string (name of your application) in the user_agent parameter.

- If needed, you can add the timeout-parameter which specifies how many seconds we will wait for a response from the service.

In [3]:
# Import the geocoding tool
from geopandas.tools import geocode

# Geocode addresses using Nominatim. Remember to provide a custom "application name" in the user_agent parameter!
geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_AB', timeout=4)

In [4]:
geo.head()

,geometry,address
0,POINT (24.91556 60.16320),"Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns..."
1,POINT (24.93166 60.16905),"Kamppi, 1, Kampinkuja, Kamppi, Helsinki, Helsi..."
2,POINT (24.94179 60.16989),"Kauppakeskus Citycenter, 8, Kaivokatu, Kluuvi,..."
3,POINT (24.97783 60.18892),"Hermannin rantatie, Verkkosaari, Kalasatama, S..."
4,POINT (24.92160 60.15665),"Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län..."


And Voilà! As a result we have a GeoDataFrame that contains our original address and a ‘geometry’ column containing Shapely Point -objects that we can use for exporting the addresses to a Shapefile for example. However, the id column is not there. Thus, we need to join the information from data into our new GeoDataFrame geo, thus making a Table Join.

#### Rate-limiting

When geocoding a large dataframe, you might encounter an error when geocoding. In case you get a time out error, try first using the timeout parameter as we did above (allow the service a bit more time to respond). In case of Too Many Requests error, you have hit the rate-limit of the service, and you should slow down your requests. To our convenience, geopy provides additional tools for taking into account rate limits in geocoding services. This script adapts the usage of geopy RateLimiter to our input data.
All in all, remember that Nominatim is not meant for super heavy use.

### Table join

Joining data between two or several tables is a common task in many (spatial) data analysis workflows. As you might remember from our earlier lessons, combining data from different tables based on common key attribute can be done easily in pandas/geopandas using the merge() -function. We used this approach in the geo-python course exercise 6.

However, sometimes it is useful to join two tables together based on the index of those DataFrames. In such case, we assume that there is same number of records in our DataFrames and that the order of the records should be the same in both DataFrames.

We can use this approach to join information from the original data to our geocoded addresses row-by-row join() -function which merges the two DataFrames together based on index by default. This approach works correctly because the order of the geocoded addresses in geo DataFrame is the same as in our original data DataFrame.

In [5]:
join = geo.join(data)
join.head()

,geometry,address,id,addr
0,POINT (24.91556 60.16320),"Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns...",1000,"Itämerenkatu 14, 00101 Helsinki, Finland"
1,POINT (24.93166 60.16905),"Kamppi, 1, Kampinkuja, Kamppi, Helsinki, Helsi...",1001,"Kampinkuja 1, 00100 Helsinki, Finland"
2,POINT (24.94179 60.16989),"Kauppakeskus Citycenter, 8, Kaivokatu, Kluuvi,...",1002,"Kaivokatu 8, 00101 Helsinki, Finland"
3,POINT (24.97783 60.18892),"Hermannin rantatie, Verkkosaari, Kalasatama, S...",1003,"Hermannin rantatie 1, 00580 Helsinki, Finland"
4,POINT (24.92160 60.15665),"Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län...",1005,"Tyynenmerenkatu 9, 00220 Helsinki, Finland"


As a result we have a new GeoDataFrame called join where we now have all original columns plus a new column for geometry. Note! If you would do the join the other way around, i.e. data.join(geo), the output would be a pandas DataFrame, not a GeoDataFrame!

Now it is easy to save our address points into a Shapefile

In [6]:
# Output file path
outfp = r"data/addresses.shp"

# Save to Shapefile
join.to_file(outfp)

### Notes about Nominatim
Nominatim works relatively nicely if you have well defined and well-known addresses such as the ones that we used in this tutorial. In practice, the address needs to exist in the OpenStreetMap database. Sometimes, however, you might want to geocode a “point-of-interest”, such as a museum, only based on it’s name. If the museum name is not on OpenStreetMap, Nominatim won’t provide any results for it, but you might be able to geocode the place using some other geocoder such as the Google Geocoding API (V3), which requires an API key. Take a look from past year’s materials where we show how to use Google Geocoding API in a similar manner as we used Nominatim here.